# Exploratory Analysis of GSE164378 Dataset (R/Seurat)

## Overview
This notebook performs a comprehensive single-cell RNA-seq analysis of the GSE164378 dataset using **Seurat** (R). This is the R equivalent of the Python/Scanpy analysis, demonstrating the same workflow in a different ecosystem.

## Dataset Source
This dataset is from the research paper:

**"Integrated analysis of multimodal single-cell data"**  
*Hao et al., Cell (2021)*  
DOI: [10.1016/j.cell.2021.04.048](https://doi.org/10.1016/j.cell.2021.04.048)  
Paper URL: https://www.cell.com/cell/fulltext/S0092-8674(21)00583-3  
GEO Accession: [GSE164378](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE164378)

## Why R/Seurat?
**Seurat** is the most widely-used R package for single-cell analysis, developed by the Satija Lab. It offers:
- Comprehensive single-cell workflows
- Excellent visualization capabilities
- Extensive documentation and community support
- Integration with the Bioconductor ecosystem
- The original WNN implementation

## Analysis Pipeline
1. **Data Loading**: Import 10X Genomics format data
2. **Quality Control**: Filter low-quality cells and genes
3. **Normalization**: Log-normalization and scaling
4. **Feature Selection**: Identify highly variable genes
5. **Dimensionality Reduction**: PCA and UMAP
6. **Clustering**: Graph-based clustering
7. **Visualization**: Cell types and marker genes

## Requirements
- R 4.0+
- Seurat, ggplot2, dplyr, patchwork
- IRkernel for Jupyter

## Note
This analysis parallels `explore_GSE164378.ipynb` (Python/Scanpy version) for direct comparison.

In [ ]:
# Load required libraries
library(Seurat)
library(ggplot2)
library(dplyr)
library(patchwork)

# Set random seed for reproducibility
set.seed(42)

# Configure plotting
options(repr.plot.width=12, repr.plot.height=8)
theme_set(theme_bw())

# Define paths
ACCESSION <- "GSE164378"
DATA_DIR <- file.path("data", ACCESSION, "suppl")
RESULTS_DIR <- file.path("results", ACCESSION)
FIGURES_DIR <- file.path(RESULTS_DIR, "figures_R")

# Create output directories
dir.create(RESULTS_DIR, recursive = TRUE, showWarnings = FALSE)
dir.create(FIGURES_DIR, recursive = TRUE, showWarnings = FALSE)

# Check dataset availability
cat("\n", strrep("=", 60), "\n")
cat("Checking dataset availability...\n")
cat(strrep("=", 60), "\n\n")

if (!dir.exists(DATA_DIR) || length(list.files(DATA_DIR, pattern = "\\.mtx\\.gz$")) == 0) {
    cat("⚠️  Dataset", ACCESSION, "not found locally.\n")
    cat("   Please run: python download_geo_dataset.py\n")
    stop("Dataset not found")
} else {
    cat("✅ Dataset", ACCESSION, "found at:", normalizePath(DATA_DIR), "\n")
    cat("   Ready to proceed with analysis!\n")
}

cat("\nConfiguration:\n")
cat("  Data directory:   ", normalizePath(DATA_DIR), "\n")
cat("  Results directory:", normalizePath(RESULTS_DIR), "\n")
cat("  Figures directory:", normalizePath(FIGURES_DIR), "\n")
cat(strrep("=", 60), "\n")

## 1. Load Data

We load the 10X Genomics format data using Seurat's `Read10X()` function. The data consists of:
- **matrix.mtx.gz**: Sparse count matrix
- **features.tsv.gz**: Gene names and IDs  
- **barcodes.tsv.gz**: Cell barcodes

We also load the metadata file containing cell-level annotations from the original study.

In [ ]:
cat("Loading 3' RNA sequencing data...\n")

# Read 10X data
# Note: Seurat's Read10X expects files without the GSM prefix
# We need to create symbolic links or specify the prefix pattern

# Load the count matrix
counts <- Read10X(
    data.dir = DATA_DIR,
    gene.column = 2,  # Use gene symbols
    cell.column = 1,
    unique.features = TRUE,
    strip.suffix = FALSE
)

cat("Count matrix loaded:", nrow(counts), "genes x", ncol(counts), "cells\n")

# Load metadata
cat("\nLoading metadata...\n")
metadata_path <- file.path(DATA_DIR, "GSE164378_sc.meta.data_3P.csv.gz")
metadata <- read.csv(gzfile(metadata_path), row.names = 1)

cat("Metadata loaded:", nrow(metadata), "cells\n")

# Find common cells
common_cells <- intersect(colnames(counts), rownames(metadata))
cat("Found", length(common_cells), "cells in both matrix and metadata\n")

# Subset to common cells
counts <- counts[, common_cells]
metadata <- metadata[common_cells, ]

# Create Seurat object
pbmc <- CreateSeuratObject(
    counts = counts,
    project = "GSE164378",
    min.cells = 3,      # Keep genes detected in at least 3 cells
    min.features = 200  # Keep cells with at least 200 genes
)

# Add metadata
pbmc <- AddMetaData(pbmc, metadata)

cat("\nSeurat object created:\n")
cat("  Cells:   ", ncol(pbmc), "\n")
cat("  Features:", nrow(pbmc), "\n")
cat("  Metadata columns:", ncol(pbmc@meta.data), "\n")

## 2. Quality Control & Filtering

Quality control metrics help identify and remove low-quality cells:

### QC Metrics
- **nFeature_RNA**: Number of genes detected per cell
- **nCount_RNA**: Total UMI counts per cell (library size)
- **percent.mt**: Percentage of mitochondrial gene expression

### Filtering Strategy
- Remove cells with < 200 genes (empty droplets)
- Remove cells with > 20% mitochondrial content (dying cells)
- Genes already filtered (detected in < 3 cells)

High mitochondrial percentage often indicates:
- Dying or stressed cells
- Poor quality samples
- Cytoplasmic RNA loss

In [ ]:
cat("\nCalculating QC metrics...\n")

# Calculate mitochondrial percentage
pbmc[["percent.mt"]] <- PercentageFeatureSet(pbmc, pattern = "^MT-")

# Display QC metrics summary
cat("\nQC Metrics Summary:\n")
cat("  Median genes per cell:  ", median(pbmc$nFeature_RNA), "\n")
cat("  Median UMIs per cell:   ", median(pbmc$nCount_RNA), "\n")
cat("  Median MT%:             ", round(median(pbmc$percent.mt), 2), "%\n")

# Visualize QC metrics
cat("\nGenerating QC plots...\n")

# Violin plots
p1 <- VlnPlot(pbmc, 
              features = c("nFeature_RNA", "nCount_RNA", "percent.mt"),
              ncol = 3,
              pt.size = 0.1) +
      plot_annotation(title = "QC Metrics Distribution")

print(p1)

# Scatter plots showing relationships
p2 <- FeatureScatter(pbmc, feature1 = "nCount_RNA", feature2 = "percent.mt") +
      ggtitle("Total UMIs vs Mitochondrial %")
p3 <- FeatureScatter(pbmc, feature1 = "nCount_RNA", feature2 = "nFeature_RNA") +
      ggtitle("Total UMIs vs Genes Detected")

print(p2 + p3)

In [ ]:
cat("\nApplying QC filters...\n")

# Store pre-filter counts
cells_before <- ncol(pbmc)

# Apply filters
pbmc <- subset(pbmc, 
               subset = nFeature_RNA > 200 & 
                       nFeature_RNA < 7500 &  # Remove potential doublets
                       percent.mt < 20)

cells_after <- ncol(pbmc)

cat("Cells before filtering:", cells_before, "\n")
cat("Cells after filtering: ", cells_after, "\n")
cat("Cells removed:         ", cells_before - cells_after, 
    "(", round(100 * (cells_before - cells_after) / cells_before, 1), "%)\n")

## 3. Normalization and Feature Selection

### Normalization
Seurat uses **log-normalization**:
1. Divide counts by total counts per cell
2. Multiply by scale factor (10,000)
3. Log-transform: log(value + 1)

This accounts for differences in sequencing depth between cells.

### Highly Variable Features (HVFs)
We identify genes with high cell-to-cell variation:
- These genes are most informative for distinguishing cell types
- Used for dimensionality reduction (PCA)
- Reduces computational burden
- Focuses on biological signal

In [ ]:
cat("\nNormalizing data...\n")

# Log-normalization
pbmc <- NormalizeData(pbmc, 
                      normalization.method = "LogNormalize",
                      scale.factor = 10000)

cat("✅ Normalization complete\n")

# Find highly variable features
cat("\nIdentifying highly variable features...\n")

pbmc <- FindVariableFeatures(pbmc,
                             selection.method = "vst",
                             nfeatures = 2000)

# Get the top 10 most variable genes
top10 <- head(VariableFeatures(pbmc), 10)
cat("Top 10 most variable genes:\n")
print(top10)

# Plot variable features
p1 <- VariableFeaturePlot(pbmc)
p2 <- LabelPoints(plot = p1, points = top10, repel = TRUE, xnudge = 0, ynudge = 0)
print(p2 + ggtitle("Highly Variable Features"))

cat("\nIdentified", length(VariableFeatures(pbmc)), "highly variable features\n")

## 4. Scaling and Dimensionality Reduction

### Scaling
Before PCA, we scale the data:
- Center each gene (mean = 0)
- Scale to unit variance (SD = 1)
- Prevents highly-expressed genes from dominating
- Optional: regress out technical variation (UMI counts, MT%)

### Principal Component Analysis (PCA)
- Linear dimensionality reduction
- Identifies axes of maximum variation
- Reduces ~2000 genes to 50 principal components
- Captures most biological signal while reducing noise

In [ ]:
cat("\nScaling data and regressing out technical effects...\n")

# Scale data and regress out technical variation
all_genes <- rownames(pbmc)
pbmc <- ScaleData(pbmc, 
                  features = all_genes,
                  vars.to.regress = c("nCount_RNA", "percent.mt"))

cat("✅ Scaling complete\n")

# Run PCA
cat("\nRunning PCA...\n")
pbmc <- RunPCA(pbmc, 
               features = VariableFeatures(object = pbmc),
               npcs = 50,
               verbose = FALSE)

cat("✅ PCA complete\n")

# Visualize PCA results
print(DimPlot(pbmc, reduction = "pca") + ggtitle("PCA: PC1 vs PC2"))

# Elbow plot to determine dimensionality
print(ElbowPlot(pbmc, ndims = 50) + 
      ggtitle("PCA Variance Explained") +
      geom_vline(xintercept = 40, linetype = "dashed", color = "red"))

cat("\n💡 Using first 40 PCs for downstream analysis (based on elbow plot)\n")

## 5. Clustering and UMAP

### Graph-Based Clustering
Seurat uses a graph-based approach:
1. **Construct KNN graph**: Connect each cell to its k nearest neighbors
2. **Find communities**: Use Louvain or Leiden algorithm
3. **Resolution parameter**: Controls granularity (higher = more clusters)

### UMAP Visualization
**UMAP** (Uniform Manifold Approximation and Projection):
- Non-linear dimensionality reduction
- Preserves local and global structure
- Projects cells into 2D for visualization
- Similar cells cluster together

In [ ]:
cat("\nConstructing neighbor graph...\n")

# Find neighbors
pbmc <- FindNeighbors(pbmc, dims = 1:40, verbose = FALSE)

cat("✅ Neighbor graph constructed\n")

# Clustering
cat("\nPerforming graph-based clustering...\n")
pbmc <- FindClusters(pbmc, 
                    resolution = 0.5,
                    algorithm = 1,  # Louvain
                    verbose = FALSE)

n_clusters <- length(unique(pbmc$seurat_clusters))
cat("✅ Identified", n_clusters, "clusters\n")

# Run UMAP
cat("\nComputing UMAP embedding...\n")
pbmc <- RunUMAP(pbmc, 
                dims = 1:40,
                verbose = FALSE)

cat("✅ UMAP complete\n")

## 6. Visualization

Now we visualize the results using UMAP plots, colored by:
1. **Seurat Clusters**: Unsupervised clustering results
2. **Cell Type Annotations**: Ground truth from the original study
3. **Marker Genes**: Expression of canonical PBMC markers

### PBMC Marker Genes
- **CD3E**: T cells (all)
- **CD4**: CD4+ T cells (helper)
- **CD8A**: CD8+ T cells (cytotoxic)
- **MS4A1** (CD20): B cells
- **GNLY**: NK cells
- **CD14**: Classical monocytes
- **FCGR3A** (CD16): Non-classical monocytes
- **PPBP**: Platelets/Megakaryocytes

In [ ]:
cat("\nGenerating UMAP visualizations...\n")

# Plot 1: Seurat clusters
p1 <- DimPlot(pbmc, 
              reduction = "umap",
              label = TRUE,
              label.size = 4,
              pt.size = 0.5) +
      ggtitle("Seurat Clusters") +
      theme(legend.position = "right")

print(p1)

# Plot 2: Cell type annotations (Level 1)
if ("celltype.l1" %in% colnames(pbmc@meta.data)) {
    p2 <- DimPlot(pbmc,
                  reduction = "umap",
                  group.by = "celltype.l1",
                  label = TRUE,
                  label.size = 3,
                  pt.size = 0.5,
                  repel = TRUE) +
          ggtitle("Cell Types (Level 1 - Ground Truth)") +
          theme(legend.position = "right")
    print(p2)
}

# Plot 3: Cell type annotations (Level 2)
if ("celltype.l2" %in% colnames(pbmc@meta.data)) {
    p3 <- DimPlot(pbmc,
                  reduction = "umap",
                  group.by = "celltype.l2",
                  label = TRUE,
                  label.size = 2,
                  pt.size = 0.5,
                  repel = TRUE) +
          ggtitle("Cell Types (Level 2 - Detailed)") +
          theme(legend.position = "none")  # Too many categories
    print(p3)
}

In [ ]:
cat("\nVisualizing marker gene expression...\n")

# Define marker genes
marker_genes <- c("CD3E", "CD4", "CD8A", "MS4A1", "GNLY", "CD14", "FCGR3A", "PPBP")

# Check which markers are available
available_markers <- marker_genes[marker_genes %in% rownames(pbmc)]
cat("Available markers:", paste(available_markers, collapse = ", "), "\n")

# Feature plots
if (length(available_markers) > 0) {
    p <- FeaturePlot(pbmc,
                     features = available_markers,
                     ncol = 4,
                     pt.size = 0.5,
                     order = TRUE) &
         theme(legend.position = "right") &
         scale_colour_gradientn(colours = viridis::viridis(100))
    
    print(p)
}

# Violin plots for markers
if (length(available_markers) >= 4) {
    p_vln <- VlnPlot(pbmc,
                     features = available_markers[1:4],
                     ncol = 2,
                     pt.size = 0.1)
    print(p_vln)
}

## 7. Summary and Comparison

### Analysis Complete! ✅

**Dataset Statistics:**
- Cells analyzed: Check `ncol(pbmc)`
- Genes: Check `nrow(pbmc)`
- Clusters identified: Check `length(unique(pbmc$seurat_clusters))`
- Cell type categories: Check metadata columns

### Python (Scanpy) vs R (Seurat) Comparison

| Aspect | Python/Scanpy | R/Seurat |
|--------|---------------|----------|
| **Normalization** | Total count + log | Log-normalization |
| **HVG Selection** | Dispersion-based | Variance stabilizing transform |
| **Scaling** | StandardScaler | Center + scale |
| **PCA** | sklearn/scipy | irlba |
| **Clustering** | Leiden (default) | Louvain (default) |
| **UMAP** | umap-learn | uwot |
| **Visualization** | matplotlib/seaborn | ggplot2 |

### Key Differences
1. **Philosophy**: 
   - Scanpy: More flexible, Pythonic
   - Seurat: More structured, R-style

2. **Performance**:
   - Scanpy: Better for very large datasets (>100k cells)
   - Seurat: Excellent for standard workflows

3. **Ecosystem**:
   - Scanpy: Integration with deep learning (PyTorch, TensorFlow)
   - Seurat: Integration with Bioconductor

4. **Visualization**:
   - Scanpy: matplotlib-based, highly customizable
   - Seurat: ggplot2-based, publication-ready defaults

### When to Use Each?
**Use Scanpy (Python) when:**
- Working with very large datasets
- Need custom algorithms
- Integration with ML/DL pipelines
- Team prefers Python

**Use Seurat (R) when:**
- Standard single-cell workflows
- Publication-quality plots needed
- Working with Bioconductor packages
- Team prefers R

### Next Steps
1. **Differential Expression**: Find marker genes for each cluster
2. **Cell Type Annotation**: Assign labels to Seurat clusters
3. **Multimodal Integration**: WNN analysis with ADT data
4. **Trajectory Analysis**: Pseudotime and differentiation paths
5. **Save Results**: Export Seurat object for future use

In [ ]:
cat("\nAnalysis complete!\n")

# Display summary statistics
cat("\nDataset Statistics:\n")
cat("  Total cells analyzed:     ", ncol(pbmc), "\n")
cat("  Total genes:              ", nrow(pbmc), "\n")
cat("  Seurat clusters:          ", length(unique(pbmc$seurat_clusters)), "\n")

if ("celltype.l1" %in% colnames(pbmc@meta.data)) {
    cat("  Cell type categories (L1):", length(unique(pbmc$celltype.l1)), "\n")
}
if ("celltype.l2" %in% colnames(pbmc@meta.data)) {
    cat("  Cell type categories (L2):", length(unique(pbmc$celltype.l2)), "\n")
}

# Optional: Save Seurat object
# Uncomment to save:
# output_file <- file.path(RESULTS_DIR, "GSE164378_seurat_processed.rds")
# cat("\nSaving Seurat object to:", output_file, "\n")
# saveRDS(pbmc, file = output_file)
# cat("✅ Saved! This file can be loaded for further analysis.\n")

cat("\n", strrep("=", 60), "\n")
cat("R/Seurat Analysis Complete!\n")
cat("Compare with Python version: explore_GSE164378.ipynb\n")
cat(strrep("=", 60), "\n")

## Citation

If you use this analysis or the GSE164378 dataset, please cite:

> Hao, Y., Hao, S., Andersen-Nissen, E., Mauck, W.M., Zheng, S., Butler, A., Lee, M.J., Wilk, A.J., Darby, C., Zager, M., et al. (2021). Integrated analysis of multimodal single-cell data. *Cell* 184, 3573-3587.e29. https://doi.org/10.1016/j.cell.2021.04.048

### Additional Resources
- **Seurat**: https://satijalab.org/seurat/
- **Seurat Tutorials**: https://satijalab.org/seurat/articles/
- **PBMC Tutorial**: https://satijalab.org/seurat/articles/pbmc3k_tutorial.html
- **Multimodal Analysis**: https://satijalab.org/seurat/articles/multimodal_vignette.html